In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc, roc_curve, roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from bayes_opt import BayesianOptimization
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier

/home/ray/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [52]:
df_main_train = pd.read_csv('../data/handled/main_train.csv', index_col=0)
df_main_test = pd.read_csv('../data/handled/main_test.csv', index_col=0)

df_bureau_train = pd.read_csv('../data/handled/bureau_train.csv', index_col=0)
df_bureau_test = pd.read_csv('../data/handled/bureau_test.csv', index_col=0)

In [53]:
df_bureau_test.index = df_bureau_test.SK_ID_CURR
df_bureau_train.index = df_bureau_train.SK_ID_CURR

In [54]:
df_bureau_train = df_bureau_train.drop(columns='SK_ID_CURR')
df_bureau_test = df_bureau_test.drop(columns='SK_ID_CURR')

In [55]:
y_train = df_main_train.TARGET

df_main_train = df_main_train.drop(columns='TARGET')
#df_main_test = df_main_test.drop(columns='TARGET')
df_bureau_train = df_bureau_train.drop(columns='TARGET')
df_bureau_test = df_bureau_test.drop(columns='TARGET')

In [56]:
df_train = pd.merge(df_main_train, df_bureau_train, left_index=True, right_index=True)
df_test = pd.merge(df_main_test, df_bureau_test, left_index=True, right_index=True)

In [57]:
df_train = pd.get_dummies(df_train, dummy_na=True)
df_test = pd.get_dummies(df_test, dummy_na=True)

In [58]:
df_train, df_test = df_train.align(df_test, join='left', fill_value=0, axis=1)

In [59]:
im = Imputer()
im.fit(df_train)
train = im.transform(df_train)
test = im.transform(df_test)

In [65]:
lr = LogisticRegression()
lr.fit(train, y_train)
res = lr.predict_proba(test)

In [61]:
test_id = df_bureau_test.index

In [62]:
def output(test_id, test_prob, sid=0):
    result = pd.DataFrame(np.column_stack((test_id, test_prob)))
    result.columns = ['SK_ID_CURR', 'TARGET']
    result['SK_ID_CURR'] = result['SK_ID_CURR'].astype('int')
    result.to_csv('submission' + str(sid) + '.csv', header=True, index=False)

In [66]:
output(test_id, res[:, 1], sid='_add_bureau')